# FNet

- Paper: Deep Learning in Searching the Spectroscopic Redshift of Quasars

- Authors: F. Rastegar Nia, M. T. Mirtorabi R. Moradi A. Vafaei.Sadr and Y.Wang


### Reference
- pipeline: [G2Net / efficientnet_b7 / baseline](https://www.kaggle.com/yasufuminakama/g2net-efficientnet-b7-baseline-training).
- dataset: https://www.kaggle.com/ywangscience/sdss-iii-iv

In [ ]:
import numpy as np
import time
import math
import random
from pathlib import Path
import pandas as pd
import os
import h5py
import sys

# ML related packages
import torch
from torch import nn, optim
import torch.nn.functional as F
import torchvision
from torch.utils.data import data, Dataset, DataLoader, TensorDataset, ConcatDataset, random_split
from torchvision import transforms
#from sklearn.metrics import roc_auc_score
#from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

# plot
import matplotlib.pyplot as plt
%matplotlib inline


## Configuration 

In [ ]:
class CFG:
    load_model = False
    best_loss = 1e10
    best_score = 1e10
    debug = False
    print_freq = 100
    num_workers = 4
    scheduler = "ReduceLROnPlateau" # CosineAnnealingLR / ReduceLROnPlateau / CosineAnnealingWarmRestarts
    model_name = "1dresnet-DR12Q-Redshift"
    epochs = 50
    T_max = 5
    lr = 0.2e-4
    min_lr = 1e-7
    batch_size = 128
    val_batch_size = 100
    weight_decay = 1e-5
    gradient_accumulation_steps = 1
    max_grad_norm = 1000 # Clipping Gradient Maximun
    factor = 0.2
    patience = 1
    eps = 1e-7
    seed = 1127802826
    n_fold = 5
    trn_fold = [0,1]  # [0, 1, 2, 3, 4]
    target_col = "target"
    train = True
    SAVEDIR = Path('./')
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Utils

In [ ]:
# ====================================================
# Utils
# ====================================================
def get_score(y_true, y_pred):
    #score = roc_auc_score(y_true, y_pred)
    score = np.mean(np.abs((y_pred-y_true)/(1+y_true)))
    return score


def init_logger(log_file=CFG.SAVEDIR / '1dresnet-DR12Q-Redshift-train.log'):
    """Save Log to file"""
    from logging import getLogger, INFO, FileHandler,  Formatter,  StreamHandler
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter("%(message)s"))
    handler2 = FileHandler(filename=log_file)
    handler2.setFormatter(Formatter("%(message)s"))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    logger.propagate = False
    return logger

LOGGER = init_logger()


def seed_torch(seed=42):
    """Random seed"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch(seed=CFG.seed)

## Load Data

In [ ]:
f = h5py.File('dr16q-log-zvi-quasar.h5', 'r') 
f.keys()

In [ ]:
def norm(x_standard, x):
    """normalize redshift 0-1"""
    x_norm = (x - np.min(x_standard))/(np.max(x_standard)-np.min(x_standard))
    return x_norm

def inv_norm(x_standard, x_norm):
    x = x_norm*(np.max(x_standard)-np.min(x_standard)) + np.min(x_standard)
    return x

In [ ]:
## read data
FLUX_NORM = f['flux_norm'][:]
Z_VI = f['z_vi'][:]
Z_QN= f['z_qn'][:]
Z_PIPE= f['z_pipe'][:]
Z_PCA= f['z_pca'][:]
Z_DR12Q= f['z_dr12q'][:]
PLATE = f['plate'][:]
MJD = f['mjd'][:]
FIBERIS = f['fiberid'][:]


## filter only DR12
Z_DR12Q_12 = Z_DR12Q[Z_DR12Q>0]
Z_PCA_12 = Z_PCA[Z_DR12Q>0]
FLUX_NORM_12 = FLUX_NORM[Z_DR12Q>0]
Z_VI_12 = Z_VI[Z_DR12Q>0]
Z_QN_12 = Z_QN[Z_DR12Q>0]
PLATE_12 = PLATE[Z_DR12Q>0]
MJD_12 = MJD[Z_DR12Q>0]
FIBERIS_12 = FIBERIS[Z_DR12Q>0]

## filter only DR16, not in DR12
Z_DR12Q_16 = Z_DR12Q[Z_DR12Q==-1]
Z_PCA_16 = Z_PCA[Z_DR12Q==-1]
FLUX_NORM_16 = FLUX_NORM[Z_DR12Q==-1]
Z_VI_16 = Z_VI[Z_DR12Q==-1]
Z_QN_16 = Z_QN[Z_DR12Q==-1]
PLATE_16 = PLATE[Z_DR12Q==-1]
MJD_16 = MJD[Z_DR12Q==-1]
FIBERIS_16 = FIBERIS[Z_DR12Q==-1]

In [ ]:
## split train and test data for original data

# entire DR16
features = torch.Tensor(FLUX_NORM).view(-1,1,4618)
labels = torch.Tensor(norm(Z_VI_12,Z_VI)).view(-1,1)
names = torch.IntTensor(list(zip(PLATE,MJD,FIBERIS)))

features_train, features_test, labels_train, labels_test, names_train, names_test \
= train_test_split(features ,labels, names, test_size= 0.1,random_state = 42 )

# only DR12
features_12 = torch.Tensor(FLUX_NORM_12).view(-1,1,4618)
labels_12 = torch.Tensor(norm(Z_VI_12,Z_VI_12)).view(-1,1)
names_12 = torch.IntTensor(list(zip(PLATE_12,MJD_12,FIBERIS_12)))

features_train_12, features_test_12, labels_train_12, labels_test_12, names_train_12, names_test_12 \
= train_test_split(features_12 ,labels_12, names_12, test_size= 0.1,random_state = 42 )

#  only in DR16, not in DR12
features_16 = torch.Tensor(FLUX_NORM_16).view(-1,1,4618)
labels_16 = torch.Tensor(norm(Z_VI_12,Z_VI_16)).view(-1,1)
names_16 = torch.IntTensor(list(zip(PLATE_16,MJD_16,FIBERIS_16)))

features_train_16, features_test_16, labels_train_16, labels_test_16, names_train_16, names_test_16 \
= train_test_split(features_16 ,labels_16, names_16, test_size= 0.1,random_state = 42 )

## Helper functions

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (remain %s)' % (asMinutes(s), asMinutes(rs))


def max_memory_allocated(device):
    MB = 1024.0 * 1024.0
    mem = torch.cuda.max_memory_allocated(device) / MB
    return f"{mem:.0f} MB"

def sdss_id(index):
    return '-'.join(map(str, index))

def get_result(result_df):
    preds = result_df['preds'].values
    labels = result_df[CFG.target_col].values
    score = get_score(labels, preds)
    LOGGER.info(f'Score: {score:<.9f}')

## Loss

In [ ]:
def z_loss(pred, label):
    #return torch.mean(torch.abs((pred-label)/(1+label)))
    return nn.MSELoss()

## Model

In [ ]:
class Residual(nn.Module):
    def __init__(self, in_channels, out_channels, use_1x1conv=False, stride=1):
        super(Residual, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=500, padding=250, stride=stride)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=200, padding=100, stride=stride)
        self.conv3 = nn.Conv1d(out_channels, out_channels, kernel_size=15, padding=6, stride=1)
        if use_1x1conv:
            self.conv4 = nn.Conv1d(in_channels, out_channels, kernel_size=1, stride=stride)
        else:
            self.conv4 = None
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.bn3 = nn.BatchNorm1d(out_channels)
        #self.fc = nn.Linear(in_features, out_features, bias=True)
        
    def forward(self, x):
        y1 = F.relu(self.bn1(self.conv1(x)))
        y2 = F.relu(self.bn2(self.conv2(y1)))
        y3 = self.bn3(self.conv3(y2))
        if self.conv4:
            x = self.conv4(x)
        return F.relu(y3+x)
    
    
def resnet_block(in_channels, out_channels, num_residuals, stride=1):
    blk = []
    for i in range(num_residuals):
        if i==0:
            blk.append(Residual(in_channels, out_channels, use_1x1conv=True, stride=stride))
        else:
            blk.append(Residual(out_channels, out_channels))
    return nn.Sequential(*blk)

def resnet():
    model = nn.Sequential()
    model.add_module('resnet_block_1', resnet_block(in_channels=1, out_channels=32, num_residuals=5, stride=1))
    model.add_module('resnet_block_2', resnet_block(in_channels=32, out_channels=64, num_residuals=1, stride=1))
    model.add_module('resnet_block_3', resnet_block(in_channels=64, out_channels=32, num_residuals=1, stride=1))
    model.add_module('resnet_block_4', resnet_block(in_channels=32, out_channels=1, num_residuals=1, stride=1))
    model.add_module('flatten', nn.Flatten())
    model.add_module('fc1', nn.Sequential(nn.Linear(in_features=4618, out_features=796, bias=True), nn.ReLU()))
    model.add_module('fc2', nn.Sequential(nn.Linear(in_features=796, out_features=199, bias=True), nn.ReLU()))
    model.add_module('fc3', nn.Linear(in_features=199, out_features=1, bias=True))
    return model

In [ ]:
model = resnet()

## Trainer

In [ ]:
def train_fn(files, model, criterion, optimizer, epoch, scheduler, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()

    # switch to train mode
    model.train()
    start = end = time.time()
    global_step = 0
    
    train_loader = DataLoader(files, 
                              batch_size=CFG.batch_size, 
                              shuffle=True, 
                              num_workers=CFG.num_workers)

    for step, d in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        x = d[0].to(device)
        labels = d[1].to(device)

        batch_size = labels.size(0)
        y_preds = model(x)
        loss = criterion(y_preds, labels)
        # record loss
        losses.update(loss, batch_size)
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        loss.backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), CFG.max_grad_norm)
        if (step + 1) % CFG.gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            global_step += 1
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0:
            output = 'Epoch: [{0}/{1}][{2}/{3}] '\
                     'Loss: {loss.val:.9f}({loss.avg:.9f}) '\
                     'Grad: {grad_norm:.9f}  '\
                     'LR: {lr:.9f}  '\
                     'Elapsed: {remain:s} '\
                     'Max mem: {mem:s}'.format(
                      epoch+1, CFG.epochs, step, len(train_loader),
                      loss=losses,
                      grad_norm=grad_norm,
                      #lr=scheduler.get_last_lr()[0],
                      lr=optimizer.param_groups[0]['lr'],
                      remain=timeSince(start, float(step + 1) / len(train_loader)),
                      mem=max_memory_allocated(CFG.device))
            #print(output)
            LOGGER.info(output)
    return losses.avg


def valid_fn(files, model, criterion, device):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    scores = AverageMeter()
    # switch to evaluation mode
    model.eval()
    filenames = []
    targets = []
    preds = []
    start = end = time.time()
    
    valid_loader = DataLoader(files, 
                              batch_size=CFG.batch_size, 
                              shuffle=True, 
                              num_workers=CFG.num_workers)
    
    for step, d in enumerate(valid_loader):
        # measure data loading time
        data_time.update(time.time() - end)
        
        targets.extend(d[1])
        filenames.extend(d[2].numpy())
        x = d[0].to(device)
        labels = d[1].to(device)

        batch_size = labels.size(0)
        # compute loss
        with torch.no_grad():
            y_preds = model(x)
            
        loss = criterion(y_preds.view(-1), labels.view(-1))
        losses.update(loss.item(), batch_size)

        preds.append(y_preds.to('cpu').numpy())
        if CFG.gradient_accumulation_steps > 1:
            loss = loss / CFG.gradient_accumulation_steps
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if step % CFG.print_freq == 0: 
            output = 'EVAL: [{0}/{1}] '\
                     'Data {data_time.val:.3f} ({data_time.avg:.3f}) '\
                     'Elapsed {remain:s} '\
                     'Loss: {loss.val:.9f}({loss.avg:.9f}) '.format(
                     step, len(valid_loader), batch_time=batch_time,
                     data_time=data_time, loss=losses,
                     remain=timeSince(start, float(step+1)/len(valid_loader)),
                     )
            LOGGER.info(output)
    predictions = np.concatenate(preds).reshape(-1)
    return losses.avg, predictions, np.array(targets), np.array(filenames).reshape(-1,3)


## Train loop

In [ ]:
# ====================================================
# Train loop
# ====================================================
def train_loop(train_files, val_files, fold=99, load_model = False):
    
    if (CFG.n_fold==1):
        LOGGER.info(f"========== training ==========")
    else:
        LOGGER.info(f"========== fold: {fold} training ==========")
    
    # ====================================================
    # scheduler 
    # ====================================================
    def get_scheduler(optimizer):
        if CFG.scheduler=='ReduceLROnPlateau':
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                             mode='min', 
                                                             factor=CFG.factor, 
                                                             patience=CFG.patience, 
                                                             verbose=True, 
                                                             eps=CFG.eps)
        elif CFG.scheduler=='CosineAnnealingLR':
            scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                                             T_max=CFG.T_max, 
                                                             eta_min=CFG.min_lr, 
                                                             last_epoch=-1)
        elif CFG.scheduler=='CosineAnnealingWarmRestarts':
            scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 
                                                                       T_0=CFG.T_0, 
                                                                       T_mult=1, 
                                                                       eta_min=CFG.min_lr, 
                                                                       last_epoch=-1)
        return scheduler

    # ====================================================
    # model & optimizer
    # ====================================================
#    if first_run:
    model.to(CFG.device)
    if load_model:
        model.load_state_dict(torch.load(CFG.SAVEDIR / f"{CFG.model_name}_fold{fold}_best_loss.pth")["model"])
        LOGGER.info(f"========== Model loaded: {CFG.model_name}_fold{fold}_best_score.pth ==========")

    optimizer = optim.Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    scheduler = get_scheduler(optimizer)
    criterion = nn.MSELoss()
    
    if load_model:
        best_score = CFG.best_score
        best_loss = CFG.best_loss
    else:
        best_score = np.inf
        best_loss = np.inf


    # ====================================================
    # loop
    # ====================================================
    
    for epoch in range(CFG.epochs):
        print("\n\n")
        start_time = time.time()
        
        # train
        avg_loss = train_fn(train_files, model, criterion, optimizer, epoch, scheduler, CFG.device)

        # eval
        avg_val_loss, preds, targets, files = valid_fn(val_files, model, criterion, CFG.device)
        files = np.array([sdss_id(x) for x in np.array(files).reshape(-1,3)])
        valid_result_df = pd.DataFrame({"target": targets, "preds": preds, "id": files})
        
        # scheduler the learning rate
        if isinstance(scheduler, optim.lr_scheduler.ReduceLROnPlateau):
            scheduler.step(avg_val_loss)
        elif isinstance(scheduler, optim.lr_scheduler.CosineAnnealingLR):
            scheduler.step()
        elif isinstance(scheduler, optim.lr_scheduler.CosineAnnealingWarmRestarts):
            scheduler.step()

        # scoring
        score = get_score(targets, preds)

        elapsed = time.time() - start_time

        LOGGER.info(f'Epoch {epoch+1} - avg_train_loss: {avg_loss:.9f}  avg_val_loss: {avg_val_loss:.9f}  time: {elapsed:.0f}s')
        LOGGER.info(f'Epoch {epoch+1} - Score: {score:.9f}')
            
        # save models of best score and best loss
        if score < best_score:
            best_score = score
            LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.9f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        CFG.SAVEDIR / f'{CFG.model_name}_fold{fold}_best_score.pth')
            
        if avg_val_loss < best_loss:
            best_loss = avg_val_loss
            LOGGER.info(f'Epoch {epoch+1} - Save Best Loss: {best_loss:.9f} Model')
            torch.save({'model': model.state_dict(), 
                        'preds': preds},
                        CFG.SAVEDIR / f'{CFG.model_name}_fold{fold}_best_loss.pth')
    
    # prediction from the model of best loss
    valid_result_df["preds"] = torch.load(CFG.SAVEDIR / f"{CFG.model_name}_fold{fold}_best_loss.pth",
                                          map_location="cpu")["preds"]

    return valid_result_df

## Training 

In [ ]:
train_files = TensorDataset(features_train_12[:], labels_train_12[:], names_train_12[:])
test_files = TensorDataset(features_test_12[:], labels_test_12[:], names_test_12[:])
all_files = TensorDataset(features_12[:], labels_12[:], names_12[:])

In [ ]:
def get_result(result_df):
    preds = result_df['preds'].values
    labels = result_df[CFG.target_col].values
    score = get_score(labels, preds)
    LOGGER.info(f'Score: {score:<.9f}')

if CFG.train:
    # train 
    LOGGER.info(f"********** Start at {time.asctime( time.localtime(time.time()) )}  **********")
    
    # training one time by setting CFG.n_fold=1
    if (CFG.n_fold==1):
        oof_df = train_loop(train_files, test_files, load_model=CFG.load_model)
        LOGGER.info(f"========== result ==========")
        get_result(oof_df)
    
    # K-Fold traning if CFG.n_fold>1
    if (CFG.n_fold>1):
        oof_df = pd.DataFrame()
        kf = KFold(n_splits=CFG.n_fold, shuffle=True, random_state=CFG.seed)
        folds = list(kf.split(all_files))
        for fold in range(CFG.n_fold):
            if fold in CFG.trn_fold:
                trn_idx, val_idx = folds[fold]
                train_files = TensorDataset(*all_files[trn_idx])
                valid_files = TensorDataset(*all_files[val_idx])
                _oof_df = train_loop(train_files, valid_files, fold, load_model=CFG.load_model)
                oof_df = pd.concat([oof_df, _oof_df])
                LOGGER.info(f"========== fold: {fold} result ==========")
                get_result(_oof_df)
        # CV result
        LOGGER.info(f"========== CV result ==========")
        get_result(oof_df)
        
    # save result
    oof_df.to_csv(CFG.SAVEDIR / f"{CFG.model_name}.csv", index=False)
    LOGGER.info(f"********** End at {time.asctime( time.localtime(time.time()) )} **********")

## Prediction

In [ ]:
fold = 99

def get_scheduler(optimizer):
    if CFG.scheduler=='ReduceLROnPlateau':
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                         mode='min', 
                                                         factor=CFG.factor, 
                                                         patience=CFG.patience, 
                                                         verbose=True, 
                                                         eps=CFG.eps)
    elif CFG.scheduler=='CosineAnnealingLR':
        scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 
                                                         T_max=CFG.T_max, 
                                                         eta_min=CFG.min_lr, 
                                                         last_epoch=-1)
    elif CFG.scheduler=='CosineAnnealingWarmRestarts':
        scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 
                                                                   T_0=CFG.T_0, 
                                                                   T_mult=1, 
                                                                   eta_min=CFG.min_lr, 
                                                                   last_epoch=-1)
    return scheduler

criterion = nn.MSELoss()

### DR12 Sample, filtered from DR16Q by Z_DR12Q>0

In [ ]:
a_12, b_12, c_12, d_12 = valid_fn(TensorDataset(features_test_12, labels_test_12, names_test_12), model, criterion, CFG.device)

In [ ]:
e_12 = 300000*((inv_norm(Z_VI_12, b_12) - inv_norm(Z_VI_12, c_12))/(1+inv_norm(Z_VI_12, c_12)))
print(abs(e_12).mean(),e_12.mean(),np.median(abs(e_12)))

In [ ]:
print(len(e_12[np.abs(e_12)<6000])/len(e_12),len(e_12[np.abs(e_12)<12000])/len(e_12),len(e_12[np.abs(e_12)<30000])/len(e_12))

### DR16 Sample, filtered from DR16Q by Z_DR12Q==-1

In [ ]:
a_16, b_16, c_16, d_16 = valid_fn(TensorDataset(features_16, labels_16, names_16), model, criterion, CFG.device)

In [ ]:
e_16 = 300000*((inv_norm(Z_VI_12, b_16) - inv_norm(Z_VI_12, c_16))/(1+inv_norm(Z_VI_12, c_16)))
print(abs(e_16).mean(),e_16.mean(),np.median(abs(e_16)))

In [ ]:
print(len(e_16[np.abs(e_16)<6000])/len(e_16),len(e_16[np.abs(e_16)<12000])/len(e_16),len(e_16[np.abs(e_16)<30000])/len(e_16))

In [ ]:
# QuasarNet
e_Q_16 = 300000*((Z_QN_16 - Z_VI_16)/(1+ Z_VI_16))
print(abs(e_Q_16).mean(),e_Q_16.mean(),np.median(abs(e_Q_16)))

In [ ]:
print(len(e_Q_16[np.abs(e_Q_16)<6000])/len(e_Q_16),len(e_Q_16[np.abs(e_Q_16)<12000])/len(e_Q_16),len(e_Q_16[np.abs(e_Q_16)<30000])/len(e_Q_16))